In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
from tqdm import tqdm
import matplotlib

dataset_path = '/vision/group/EPIC-KITCHENS/'
annotations_foldername = 'annotations'
annotations_folderpath = os.path.join(dataset_path, annotations_foldername)
visual_dataset_path = os.path.join(dataset_path, 'EPIC_KITCHENS_2018')
visual_images_foldername = 'object_detection_images'
visual_images_folderpath = os.path.join(visual_dataset_path, visual_images_foldername)

# training data
training_action_labels = 'EPIC_train_action_labels.csv'
training_object_labels = 'EPIC_train_object_labels.csv'
train_action_csvpath = os.path.join(annotations_folderpath, training_action_labels)
train_object_csvpath = os.path.join(annotations_folderpath, training_object_labels)

class_key = 'EPIC_noun_classes.csv'
class_key_csvpath = os.path.join(annotations_folderpath, class_key)

assert os.path.exists(train_action_csvpath), "{} does not exist".format(train_action_csvpath)
assert os.path.exists(train_object_csvpath), "{} does not exist".format(train_object_csvpath)
assert os.path.exists(visual_images_folderpath), "{} does not exist".format(visual_images_folderpath)
assert os.path.exists(class_key_csvpath), "{} does not exist".format(class_key_csvpath)

action_df = pd.read_csv(train_action_csvpath)
object_df = pd.read_csv(train_object_csvpath)
class_key_df = pd.read_csv(class_key_csvpath)

In [7]:
class_key_dict = dict(zip(class_key_df.class_key, class_key_df.noun_id))

In [8]:
class_key_dict

{'Nothing': 0,
 'pan': 1,
 'pan:dust': 2,
 'tap': 3,
 'plate': 4,
 'knife': 5,
 'bowl': 6,
 'spoon': 7,
 'cupboard': 8,
 'drawer': 9,
 'fridge': 10,
 'lid': 11,
 'hand': 12,
 'onion': 13,
 'onion:spring': 14,
 'pot': 15,
 'glass': 16,
 'water': 17,
 'fork': 18,
 'board:chopping': 19,
 'bag': 20,
 'sponge': 21,
 'spatula': 22,
 'cup': 23,
 'oil': 24,
 'bin': 25,
 'meat': 26,
 'potato': 27,
 'bottle': 28,
 'container': 29,
 'tomato': 30,
 'salt': 31,
 'cloth': 32,
 'sink': 33,
 'door:kitchen': 34,
 'pasta': 35,
 'dish:soap': 36,
 'food': 37,
 'kettle': 38,
 'box': 39,
 'carrot': 40,
 'sauce': 41,
 'colander': 42,
 'milk': 43,
 'rice': 44,
 'garlic': 45,
 'pepper': 46,
 'hob': 47,
 'dough': 48,
 'dishwasher': 49,
 'egg': 50,
 'cheese': 51,
 'bread': 52,
 'table': 53,
 'salad': 54,
 'microwave': 55,
 'oven': 56,
 'cooker:slow': 57,
 'coffee': 58,
 'filter': 59,
 'jar': 60,
 'rack:drying': 61,
 'chicken': 62,
 'tray': 63,
 'mixture': 64,
 'towel': 65,
 'towel:kitchen': 66,
 'peach': 67,
 's

In [45]:
hierarchy = {'vegetables': ['onion', 'potato', 'tomato', 'garlic', 'salad',
               'carrot', 'leaf', 'vegetable', 'lettuce', 'courgette',
               'mushroom', 'cucumber', 'squash', 'corn', 'aubergine', 
                'paste:garlic','avocado', 'leek', 'onion:spring', 'cabbage',
                'bean:green', 'celery', 'spinach', 'broccoli', 'pea', 'kale',
                'sprout', 'artichoke', 'asparagus'],
            'fruits_nuts': ['peach', 'olive', 'nut:pine', 'kiwi', 'banana', 
                            'lemon', 'lime', 'grape', 'blueberry', 'melon', 
                            'pear', 'raisin', 'coconut', 'apple', 'berry', 
                            'fruit', 'cherry', 'almond', 'pineapple'],
            'dairy_eggs': ['cheese', 'egg', 'milk', 'yoghurt', 'cream', 'butter'],
            'meat': ['meat', 'chicken', 'fish', 'sausage', 'bacon', 'burger:tuna',
                    'tuna', 'salami', 'salmon', 'beef', 'stick:crab', 'shrimp', 
                     'turkey'],
            'spices_herbs_sauces': ['sauce', 'spice', 'colander', 'pepper', 'salt', 
                                    'curry', 'ginger', 'chilli', 'pesto', 'oregano',
                                   'sugar', 'powder:coconut', 'parsley', 'basil',
                                   'seed', 'mint', 'coriander', 'thyme', 'dressing:salad',
                                   'paprika', 'leaf:mint', 'cumin', 'flake:chilli', 'grass:lemon',
                                   'caper', 'mayonnaise', 'herb', 'cinnamon', 'tahini', 'turmeric',
                                   'syrup', 'mustard', 'syrup', 'mustard', 'ketchup', 
                                    'pepper:cayenne', 'garni:bouquet'], 
            'liquids': ['oil', 'coffee', 'liquid', 'tea', 'water', 'vinegar', 'wine', 'smoothie', 
                        'juice', 'mocha', 'beer', 'nesquik', 'coke', 'drink'],
            'other ingredients': ['pasta', 'rice', 'skin', 'mixture', 'tofu', 'cereal', 
                                  'ingredient', 'honey', 'stock', 'noodle', 'oat', 'roll',
                                  'leftover', 'nutella', 'oatmeal', 'hummus', 'chip', 'dumpling',
                                  'ring:onion', 'breadcrumb', 'popcorn', 'butter:peanut', 
                                  'bar:cereal', 'chocolate', 'crisp'],
            'baked_baking': ['dough', 'bread', 'tortilla', 'pizza', 'flour', 
                             'biscuit', 'pie', 'quorn', 'yeast', 'cake', 'pancake', 'croissant',
                             'muffin'], #dough ball
            'cooked food': ['food', 'risotto', 'omelette', 'soup', 'sandwich', 'paella', 'fishcakes',
                           'mat:sushi'],
            'packaging': ['package', 'bag', 'wrap', 'paper', 'foil', 'wrap:plastic', 'capsule', 
                          'scrap'],
            'containers': ['box', 'container', 'bottle', 'jar', 'jug', 'cover', 'can', 'basket',
                            'cap', 'support', 'backpack', 'cork', 'support', 'backpack', 'cork'],
            'cutlery': ['knife', 'spoon', 'spatula', 'fork', 'cutlery', 'chopstick', 'ladle'],
            'kitchenware': ['pan', 'plate', 'bowl', 'pot', 'cup', 'lid', 'board:chopping', 'glass',
                            'tray', 'filter', 'scissors', 'brush', 'peeler:potato', 'strainer', 
                            'utensil', 'tongs', 'pestle', 'mortar','grater','grinder', 'base', 
                            'pin:rolling', 'holder:filter', 'time', 'presser', 'timer', 'slicer', 
                            'clip', 'whetstone', 'cutter:pizza', 'juicer:lime',  'mesh', 'shaker:pepper', 
                           'squeezer:lime', 'rubber', 'opener:bottle', 'masher'],
            'appliances': ['tap', 'fridge', 'hob', 'oven', 'kettle', 'maker:coffee', 'dishwasher',
                            'microwave', 'machine:washing', 'processor:food', 'grill', 'freezer',
                          'scale', 'heater', 'cooker:slow', 'toaster', 'button', 'blender', 'heat', 
                          'plug', 'phone', 'watch', 'fan:extractor', 'flame', 'tab', 'wire', 
                           'control:remote', 'alarm', 'power', 'stereo', 'fire'],
            'furniture': ['cupboard', 'drawer', 'sink', 'top', 'rack:drying', 'table', 'kitchen',
                         'mat', 'shelf', 'floor', 'switch', 'chair', 'desk', 'candle', 'boiler', 'light',
                         'handle', 'knob', 'mouse', 'door:kitchen', 'rug', 'tablet', 'window', 'stand'],
            'fabric_clothes': ['cloth', 'clothes', 'glove:oven', 'towel', 'napkin', 'towel:kitchen', 
                               'tablecloth', 'shirt', 'apron', 'poster', 'tissue', 'label', 'boxer',
                              'guard:hand', 'sock', 'sticker', 'trousers', 'sleeve', 'sheets'],
            'trash': ['bin', 'rubbish', 'shell:egg', 'sponge', 'liquid:washing', 'soap', 'powder:washing',
                     'spot', 'pan:dust'],
            'human': ['finger'],
            'uncategorized': ['content', 'rest', 'part', 'form', 'supplement', 'dust', 'instruction', 
                              'recipe', 'air']}

def extend_all(l):
    a = []
    for i in l:
        a.extend(i)
    return a

for element in extend_all(hierarchy.values()):
    assert element in class_key_dict, "{} not in class_key_dict".format(element)
    

In [48]:
print(set(extend_all(hierarchy.values())) - set(class_key_dict.keys()))
print(set(class_key_dict.keys()) - set(extend_all(hierarchy.values())))

set()
{'slipper', 'hand', 'gravy', 'trouser', 'hat', 'wall', 'lighter', 'tube', 'alcohol', 'straw', 'pith', 'rim', 'cd', 'waffle', 'Nothing', 'dish:soap', 'tail', 'rosemary', 'driver:screw', 'jambalaya', 'ball', 'ladder', 'remover:spot', 'heart', 'casserole', 'envelope', 'lamp', 'funnel', 'lead', 'tarragon', 'whisk', 'book', 'rinse'}


In [ ]:
# slipper -> fabric_clothes
# hand -> human
# gravy -> spices_herbs_sauces
# trouser -> fabric_clothes
# hat -> fabric_clothes
# wall -> furniture
# lighter -> appliances
# tube -> appliances
# alcohol -> liquids
# straw -> appliances
# pith -> trash
# rim -> uncategorized
# cd -> uncategorized
# waffle -> baked_baking
# Nothing -> uncategorized